In [61]:
import re
frs_match = re.compile(r"^(?:([\*]*) )?([\+\* ]*)(FRS\d+)\.?([\d\.]+)?([\+\*\:]*)?\s*(.+)?$").match
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match

class NoMatch(Exception):
    pass

class NumState():
    VALID_INDENTS = "*+"
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = self._newstate()
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
    def _newstate(self):
        return [self._startval]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
    
    def set_state(self, lvl, val):
        self._grow(lvl)
        self.state[lvl] = val
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def set_str(self, s):
        m = frs_match(s)
        if not m:
            raise NoMatch(s)
        return self.set_match(m)

    def set_match(self, m):
        indent, pre_fmt, frs, nums, post_fmt, bob = m.groups()
        self.ilvl = len(indent or "")
        self.fmt = pre_fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def feed(self, line):
        self.line = line
        m = frs_match(line)
        if m:
            self.set_match(m)
            return self.LINE_FRS
        m = subitem_match(line)
        if m:
            self.set_indent(len(m.group(1)))
            return self.LINE_SUBITEM
        return self.LINE_OTHER
    
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
        
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def parse_text(self, text):
        lines = text.splitlines()
        for line in lines:
            typ = self.feed(line)
            if typ == self.LINE_FRS:
                v = line
            elif typ == self.LINE_SUBITEM:
                fmt = self.frs_string()
                if not fmt:
                    v = line
                    break
                try:
                    _, tail = line.split(" ", 1)
                except ValueError:
                    v = fmt
                else:
                    v = " ".join((fmt, tail))
            elif typ == self.LINE_OTHER:
                v = line
            else:
                raise ValueError(typ)
            yield v

In [236]:
import re
frs_match = re.compile(r"^(?:([\*]*) )?([\+\* ]*)(FRS\d+)\.?([\d\.]+)?([\+\*\:]*)?\s*(.+)?$").match
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match
header_match = re.compile(r"^(?:([\*\+]*) )?([\+\*]+)([^\*\+]+?)(\:?[\+\*]+\:?)\s*(.+)?$").match
frs_match2 = re.compile(r"(FRS\d+)\.?([\d\.]+)?").match
num_escape_match = re.compile(r"^(\*{2})(\#+)").match

class FRSParser():
    VALID_INDENTS = "*+"
    
    # inputs
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    LINE_NUM_ESCAPE = 4
    
    # states
    ST_INIT = 0
    ST_NONUM = 1
    ST_FRS = 2
    
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = []
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
        self.sm_st = self.ST_INIT
        self.sm = [
            # ST_INIT             ST_NONUM             ST_FRS
            [self.set_frs,        self.set_frs,        self.set_frs],        # LINE_FRS
            [None,                None,                self.set_subitem],    # LINE_SUBITEM
            [None,                None,                None],                # LINE_OTHER
            [self.set_header,     self.set_header,     self.set_header],     # LINE_HEADER
            [self.set_num_escape, self.set_num_escape, self.set_num_escape], # LINE_NUM_ESCAPE
        ]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
        
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
    
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def feed(self, line):
        typ, arg = self._feed(line)
        func = self.sm[typ][self.sm_st]
        if func:
            st, rv = func(arg)
            self.sm_st = st
            
            return rv
        else:
            return line
    
    def _feed(self, line): 
        m = header_match(line)
        if m:
            ind, fmt, s, _, _ = m.groups()
            m2 = frs_match2(s)
            if m2:
                typ = self.LINE_FRS
                frs, nums = m2.groups()
                arg = (line, ind, fmt, frs, nums)
            else:
                typ = self.LINE_HEADER
                arg = (line, ind, fmt, s)
            return typ, arg
        m = subitem_match(line)
        if m:
            typ = self.LINE_SUBITEM
            arg = line, len(m.group(1) or "")
            return typ, arg
        m = num_escape_match(line)
        if m:
            typ = self.LINE_NUM_ESCAPE
            arg = line,
            return typ, arg
        return self.LINE_OTHER, (line,)
    
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def parse_text(self, text):
        for line in text.splitlines():
            yield self.feed(line)
    
    # State machine methods
                                
    def set_frs(self, arg):
        line, indent, fmt, frs, nums = arg
        self.ilvl = len(indent or "")
        self.fmt = fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        return self.ST_FRS, line
    
    def set_header(self, arg):
        return self.ST_NONUM, arg[0]
    
    def set_subitem(self, arg):
        line, n = arg
        self.set_indent(n)
        fmt = self.frs_string()
        try:
            _, tail = line.split(" ", 1)
        except ValueError:
            v = fmt
        else:
            v = " ".join((fmt, tail))
        return self.sm_st, v
    
    def set_num_escape(self, arg):
        line, = arg
        return self.sm_st, line.lstrip("*")
    
NumState = FRSParser

In [237]:
def assert_ab(a, b):
    try:
        assert a == b, "%r != %r" % (a, b)
    except AssertionError as e:
        print(a.splitlines())
        print(b.splitlines())
        raise

def test_match(s):
    return frs_match(s).groups()

def assert_match(s, exp):
    m = test_match(s)
    assert m == exp, (m, exp)
    
assert_match("*FRS1234*", (None, "*", "FRS1234", None, "*", None))
assert_match("* *FRS1234*", ("*", "*", "FRS1234", None, "*", None))
assert_match("*FRS1234.5.6*", (None, "*", "FRS1234", "5.6", "*", None))
assert_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234", "5.6", "+*", None))
assert_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234", "5.6.7.8", "+*", None))
assert_match("* *FRS1234:*", ("*", "*", "FRS1234", None, ":*", None))
assert_match("* *FRS1234*:", ("*", "*", "FRS1234", None, "*:", None))
assert_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", None, "*:", "Bob"))

def assert_header_match(s, exp):
    assert_ab(header_match(s).groups(), exp)
    
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("*URS*", (None, "*", "URS", "*", None))
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*", ("*", "*", "FRS1234", "*", None))
assert_header_match("*FRS1234.5.6*", (None, "*", "FRS1234.5.6", "*", None))
assert_header_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234.5.6", "+*", None))
assert_header_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234.5.6.7.8", "+*", None))
assert_header_match("* *FRS1234:*", ("*", "*", "FRS1234", ":*", None))
assert_header_match("* *FRS1234*:", ("*", "*", "FRS1234", "*:", None))
assert_header_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", "*:", "Bob"))
assert header_match("**") == None
assert header_match("***") == None

def assert_num_escape(s, exp):
    m = num_escape_match(s)
    r = m.groups() if m else None
    assert_ab(r, exp)
    
assert_num_escape("**# Bob", ("**", "#"))
assert_num_escape("**## Bob", ("**", "##"))
assert_num_escape("**### Bob", ("**", "###"))
assert_num_escape("# Bob", None)
assert_num_escape("## Bob", None)
assert_num_escape("### Bob", None)
assert_num_escape("* Bob", None)
assert_num_escape("** Bob", None)
assert_num_escape("*** Bob", None)

def set_str(st, s):
    ind, fmt, fs, _, _ = header_match(s).groups()
    frs, nums = frs_match2(fs).groups()
    st.set_frs((s, ind, fmt, frs, nums))
    

def test_state(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1*", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1*", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    set_str(st, s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    set_str(st, s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    set_str(st, s)
    st.sm_st = st.ST_FRS
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("*FRS1234.1*", "# ", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## ", "** *FRS1234.1.1.1:*")
assert_feed_indent("*FRS1234.1*", "#", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "##", "** *FRS1234.1.1.1:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

def assert_parse_text(txt, exp):
    st = NumState()
    lines = list(st.parse_text(txt))
    res = "\n".join(lines)
    assert_ab(res, exp)

lines = """*FRS1234.1:*
#"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*"""
assert_parse_text(lines, exp)

lines="""*FRS1234.1:*
#
## Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
#
##
"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*
** *FRS1234.1.1.1:* Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
* *FRS1234.2.1:*
** *FRS1234.2.1.1:*"""

assert_parse_text(lines, exp)

lines = "+*FRS*+\n*FRS1234:*\n#"
exp = "+*FRS*+\n*FRS1234:*\n* *FRS1234.1:*"
assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
*** Bob
## Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
*** Bob
** *FRS2908.1.1.1:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
#### Biff
*** Bob
# Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
**** *FRS2908.1.1.1.1.1:* Biff
*** Bob
* *FRS2908.1.2:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.0*
# Bob1
## Bob2
*** Fred1
*** Fred2
## Bob3
## Bob4
## Bob5
*** Fred3
#### Bob6
#### Bob7
#### Bob8
*** Fred4
# Bob9
"""

exp = """*FRS2908.0*
* *FRS2908.0.1:* Bob1
** *FRS2908.0.1.1:* Bob2
*** Fred1
*** Fred2
** *FRS2908.0.1.2:* Bob3
** *FRS2908.0.1.3:* Bob4
** *FRS2908.0.1.4:* Bob5
*** Fred3
**** *FRS2908.0.1.4.1.1:* Bob6
**** *FRS2908.0.1.4.1.2:* Bob7
**** *FRS2908.0.1.4.1.3:* Bob8
*** Fred4
* *FRS2908.0.2:* Bob9"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** Fred:
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** Fred:
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

def assert_parse_empty(s, exp):
    st = NumState()
    res = None
    for res in st.parse_text(s):
        pass
    assert_ab(res, exp)
    
assert_parse_empty("*User Requirements*\n# Bar", "# Bar")

In [245]:
lines="""
*User Requirements*
# Users performing multiple standard processes (i.e. for different clients) should have the ability to configure different System Variables files in the LabVIEW Data folder, with different names, for each process.  When starting a process, they should be able to load one of these pre-set files, knowing that it matches what they expect it to report.
# A 'Revert' button must function similarly to those in the Alarms and Logger editors

Settings Editor:
!Settings_Editor.png!
Numbered Mockup:
!Settings_Editor_Numbered.png!

*FRS2926.0* Appearances
* Item 1 is a file path indicator
* Item 2 is a Booolean button labeled Open
* Item 3 is a Booolean button labeled Save
* Item 4 is a Booolean button labeled Make Active
* Item 5 is a Booolean button labeled Revert
* Item 6 is a 1-D array of cluster of four elements:
** A string indicator
** A numeric control
** A boolean indicator
** A string indicator

*FRS2926.1* Functionality
# When first navigating to the screen: 
## Item 1 shall be blank
## Item 2 shall be enabled
## Items 3-5 shall be disabled and grayed out
## When item 2 is clicked, an open file dialog shall appear per FRS2819.0-2819.1
# When a file has been opened through the open file dialog:
## Item 1 shall contain the name of that file
## Items 2-6 shall be enabled
## When item 2 is clicked, an open file dialog shall appear per FRS2819.0-2819.1
## When item 3 is clicked, a save file dialog shall appear per FRS2819.0-2819.1
## When item 4 is clicked, the current file as displayed in item 6 shall be made the active logger settings file on the RIO. 
## When item 5 is clicked, the currently opened file as indicated in item 1 shall be reloaded from disk. 
## Item 6 shall display the contents of the currently opened file, in columns as specified above. 
## Clicking a row in the second column of item 6 shall 

## Only the "Value" column 
# Item 1 will display the path of the current config file the user is editing
## It is updated when the user opens or saves a file
# Clicking on Item 2 will open a file dialog allowing the user to select a valid System Variables.
## If the user press cancel, return to the previous state
# Clicking on Item 3 will open a file dialog allowing the user to save the current System Variables file as it is or as a different file
## If the user press cancel, return to the previous state
## If the user saves the file as System Variables.cfg, it will automatically be made active on the RIO
# Clicking on Item 4 will make the current file active on the RIO
## It will also save the current configuration to System Variables.cfg to sync the RIO and Atom configuration files.
## A user event will be generated with the following contents: "Made the following system variable file active on the RIO: (carriage return) (file content in xml format)."
# Clicking on Item 5 will allow the user to revert back to the current configuration file
# When user clicks on Item 2 and 5 and there unsaved changes, a prompt is displayed asking the user if they want to continue or cancel
## Clicking continue performs the user action.
## Clicking cancel returns the program to the previous state.
# Users can change the Value column in Item 6. Settings and Group columns must NOT be changed.
"""
res = "\n".join(NumState(3).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*User Requirements*
# Users performing multiple standard processes (i.e. for different clients) should have the ability to configure different System Variables files in the LabVIEW Data folder, with different names, for each process.  When starting a process, they should be able to load one of these pre-set files, knowing that it matches what they expect it to report.
# A 'Revert' button must function similarly to those in the Alarms and Logger editors

Settings Editor:
!Settings_Editor.png!
Numbered Mockup:
!Settings_Editor_Numbered.png!

*FRS2926.0* Appearances
* Item 1 is a file path indicator
* Item 2 is a Booolean button labeled Open
* Item 3 is a Booolean button labeled Save
* Item 4 is a Booolean button labeled Make Active
* Item 5 is a Booolean button labeled Revert
* Item 6 is a 1-D array of cluster of three elements:
** A string indicator labeled Setting
** A numeric control labeled Value
** A string indicator labeled Group

*FRS2926.1* Functionality
* *FRS2926.1.1:* When fi

In [242]:
res = "\n".join(NumState(3).parse_text(clipboard.paste()))
print(res)
import clipboard
clipboard.copy(res)

*FRS2924.1* Shell UI Actions in response to the "Shutdown" Server call:
* *FRS2924.1.1:* When mode=turnoff, the shell shall execute the Shutdown case of the Power sub VI (issue #2922)
* *FRS2924.1.2:* When mode = reboot, the shell shall execute the Reboot case of the Power sub VI (issue #2922)
* *FRS2924.1.3:* When mode = anything else, the shell shall execute the Default case of the Power sub VI (issue #2922)

*Notes:*
The 'Switch Windows User' case of that sub VI should never be executed. 
